## **Librerías**

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
pd.options.display.max_columns = False

In [3]:
from datetime import datetime, timedelta
import winsound as cst

In [4]:
import sys
sys.path.append('./')
sys.path.append('../')

In [5]:
import Utils.consultas as cst 

In [ ]:
#query ultimo registro segmentado
query=f'''
select operador,rango_ult_registro,count(*) from (
select numero,
       ult_registro,
       case
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 <1 then '<=1'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 2 and 3 then '1 y 3'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 4 and 6 then '3 y 6'
            else '>6'
       end as rango_ult_registro,
       operador 
from JMARTINEZ_ANA.TBL_NUMEROS_PNN
) group by operador,rango_ult_registro
order by operador

'''
ult_registro=cst.consulta(query)


In [ ]:
#query total protaciones
query=f'''
with base as (
select numero,
       ult_registro,
       case
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 <1 then '<=1'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 1 and 3 then '1 y 3'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 3 and 6 then '3 y 6'
            else '>6'
       end as rango_ult_registro,
       operador 
from JMARTINEZ_ANA.TBL_NUMEROS_PNN),
portaciones_ext as (
select port_number,count(*) as port_ext
from YVANEGAS_PLAC.PORT_PLAY_AS_OTHERS
where port_number in (
select numero from JMARTINEZ_ANA.TBL_NUMEROS_PNN
)group by port_number
),
portaciones_int as (
select msisdn,count(*) as port_int
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING 
WHERE  MOVIMIENTO_TIPO_NOMBRE IN ('GROSS ADDS PORTACION','CHURN PORTACION')
and msisdn in (
select numero from JMARTINEZ_ANA.TBL_NUMEROS_PNN
) group by msisdn 
)


select operador,portaciones,count(*) from (
select a.numero,a.operador,
       case 
              when  coalesce(b.port_ext,0)+coalesce(c.port_int,0) =0 then 'no hay 0 '
              when  coalesce(b.port_ext,0)+coalesce(c.port_int,0) =1 then ' solo 1 '
              when  coalesce(b.port_ext,0)+coalesce(c.port_int,0) between 2 and 3 then '1 y 3 '
              when  coalesce(b.port_ext,0)+coalesce(c.port_int,0) >3 then 'mas de 3 '
       end as portaciones
from base a 
left join portaciones_ext b on a.numero=b.port_number
left join portaciones_int c on a.numero=c.msisdn
)group by operador,portaciones
order by operador

'''
portaciones=cst.consulta(query)

In [ ]:
#query ultima portacion rango
query=f'''
with base as (
select numero,
       ult_registro,
       case
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 <1 then '<=1'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 1 and 3 then '1 y 3'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-to_date(ult_registro,'yyyy/mm/dd'))/30 between 3 and 6 then '3 y 6'
            else '>6'
       end as rango_ult_registro,
       operador 
from JMARTINEZ_ANA.TBL_NUMEROS_PNN),
portaciones_ext as (
select port_number,max(created_date) as ult_portacion_ext
from YVANEGAS_PLAC.PORT_PLAY_AS_OTHERS
where port_number in (
select numero from JMARTINEZ_ANA.TBL_NUMEROS_PNN
)group by port_number
),
portaciones_int as (
select msisdn,to_date(max(tiempo_dk),'yyyy/mm/dd') as ult_portacion_int
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING 
WHERE  MOVIMIENTO_TIPO_NOMBRE IN ('GROSS ADDS PORTACION','CHURN PORTACION')
and msisdn in (
select numero from JMARTINEZ_ANA.TBL_NUMEROS_PNN
) group by msisdn 
)
select operador,rango_ult_portacion,count(*) from (
select numero,operador,
       case
            when  to_number(to_date(20240301,'yyyy/mm/dd')-ult_portacion)/30 <1 then '<=1'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-ult_portacion)/30 between 1 and 3 then '1 y 3'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-ult_portacion)/30 between 3 and 6 then '3 y 6'
            when  to_number(to_date(20240301,'yyyy/mm/dd')-ult_portacion)/30 between 6 and 12 then '6 y 12'
            else '>12'
       end as rango_ult_portacion
from (
select a.numero,a.operador,
       ult_portacion_int,
       ult_portacion_ext,
       case 
              when coalesce(ult_portacion_ext,to_date(20200101,'yyyy/mm/dd'))>coalesce(ult_portacion_int,to_date(20200101,'yyyy/mm/dd')) then ult_portacion_ext
              when coalesce(ult_portacion_ext,to_date(20200101,'yyyy/mm/dd'))<coalesce(ult_portacion_int,to_date(20200101,'yyyy/mm/dd')) then ult_portacion_int
       end as ult_portacion
from base a 
left join portaciones_ext b on a.numero=b.port_number
left join portaciones_int c on a.numero=c.msisdn
where ult_portacion_ext is not null or ult_portacion_int is not null
)
)
group by operador,rango_ult_portacion
order by operador 
'''
ult_portacion=cst.consulta(query)

In [ ]:
query=f'''

'''

In [10]:
prueba.to_csv('upselling_popup.csv')

## **Variables Globales**

In [6]:
dia = int(str(datetime.now()).replace('-', '')[:8])
mes = int(str(datetime.now()).replace('-', '')[:6])

## **Datos**

## **Limpieza**

## **Analísis y Exploración**